# 🏠 DeepRoof-2026: Multi-Task Training Notebook

Welcome to the official training environment for the **DeepRoof-2026 AI Roof Layout Engine**. 

This notebook allows you to:
1. **Visualize** the OmniCity dataset labels (Instance Masks + Surface Normals).
2. **Configure** training parameters for either **Scratch Training** or **Fine-Tuning**.
3. **Launch** the high-performance training loop optimized for A100 GPUs.
4. **Evaluate** and visualize model predictions on new satellite imagery.

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from mmengine.config import Config
from mmengine.runner import Runner
from pathlib import Path

# Check GPU Status
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

## 📂 1. Dataset Preview

Before training, let's look at what our model will see. We combine **Satellite View 1** images with **Instance Masks** (segmentation) and **Surface Normals** (geometry).

In [ ]:
def preview_dataset(data_root, num_samples=3):
    data_path = Path(data_root)
    sample_ids = []
    with open(data_path / 'train.txt', 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    
    for i, sid in enumerate(sample_ids):
        img = cv2.imread(str(data_path / 'images' / (sid + '.jpg')))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(str(data_path / 'masks' / (sid + '.png')), cv2.IMREAD_UNCHANGED)
        # Colorize mask for better visibility
        mask_vis = (mask % 20) * 12 # Simple color recycling
        mask_vis = cv2.applyColorMap(mask_vis.astype(np.uint8), cv2.COLORMAP_JET)
        
        normals = np.load(str(data_path / 'normals' / (sid + '.npy')))
        normals_vis = ((normals + 1) * 127.5).astype(np.uint8)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(f"Image: {sid}"); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Instance Mask"); axes[i, 1].axis('off')
        axes[i, 2].imshow(normals_vis); axes[i, 2].set_title("Surface Normals"); axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Change this path to your prepared dataset location
preview_dataset("../data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration

### 📊 Hyperparameter Overview
| Parameter | Value | Rationale |
| :--- | :--- | :--- |
| **Resolution** | 1024x1024 | Highest detail for complex roof layouts. |
| **Duration** | 20,000 iters | ~16 Epochs (Ideal for fine-tuning without overfitting). |
| **Batch Size** | 4 per GPU | optimized for A100 40GB/80GB memory. |
| **Optimization** | AMP + AdamW | Mixed precision for 2x speedup on A100. |
| **Task** | Multi-Task | Learns segmentation + geometry simultaneously. |

In [ ]:
MODE = "fine-tune" # Options: "fine-tune" or "scratch"
CONFIG_FILE = "../configs/deeproof_finetune_swin_L.py"
WORK_DIR = "../work_dirs/swin_l_omnicity_v2"

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR

# Overwrite some settings based on notebook preferences
cfg.train_dataloader.batch_size = 4
cfg.train_cfg.max_iters = 20000

if MODE == "scratch":
    cfg.load_from = None
    cfg.optimizer.lr = 0.0001 # Higher LR for training from scratch
    print("🚀 Configured for Training from Scratch")
else:
    print(f"🎯 Configured for Fine-tuning with weights: {cfg.load_from}")

# Setup Checkpoint Hooks (Save best mIoU)
cfg.default_hooks.checkpoint = dict(
    type='CheckpointHook', 
    by_epoch=False, 
    interval=2000, 
    save_best='mIoU', 
    rule='greater'
)

print("✅ Configuration Validated.")

## 🚀 3. Start Training

This cell will initialize the MMSeg Runner and start the training process. Training logs will be displayed in real-time.

In [ ]:
# Ensure custom modules are registered
from mmengine.registry import MODELS, DATASETS
# Custom imports are handled by the config, but we can double check here
print(f"Registered Models: {len(MODELS.module_dict)}")

runner = Runner.from_cfg(cfg)
runner.train()

## 🔍 4. Visualize Prediction

After training, let's run the model on a validation image to see the roof segmentation and geometry output.

In [ ]:
from mmseg.apis import init_model, inference_model

# Load the best checkpoint generated during training
CHECKPOINT = os.path.join(WORK_DIR, 'best_mIoU.pth')

if os.path.exists(CHECKPOINT):
    model = init_model(CONFIG_FILE, CHECKPOINT, device='cuda:0')
    
    # Inference
    img_path = "../data/OmniCity/images/some_sample.jpg" # Update with actual valid path
    if os.path.exists(img_path):
        result = inference_model(model, img_path)
        # The model returns segmentation and attached normals
        # Visualization logic here...
        print("Prediction Complete.")
else:
    print("No checkpoint found. Please complete training first.")